In [10]:
# -*- coding:utf-8 -*-
import torch
import torch.nn as nn
import torchvision.datasets as dsets
import torchvision.transforms as transforms
from torch.autograd import Variable
import scipy.io as sio
import h5py
import numpy as np
import torch.utils.data as Data
import os
import torch
import torch.utils.data as data
from PIL import Image
import operator 
from functools import reduce
import  xml.dom.minidom
types = ["dry","mid","mix","oil"]
min_width = 2000
min_height = 2000
for type in types:
    filelist = []
    xmllist = []
    for filename in os.listdir(r"./"+type):
        if filename.endswith(".jpg"):
            filelist.append(filename)
        else:
            xmllist.append(filename)
    for i in range(len(filelist)):
        im = Image.open(type+"/"+filelist[i])
        dom = xml.dom.minidom.parse('./'+type+'/'+xmllist[i])
        root = dom.documentElement
        xmins = root.getElementsByTagName('xmin')
        ymins = root.getElementsByTagName('ymin')
        xmaxs = root.getElementsByTagName('xmax')
        ymaxs = root.getElementsByTagName('ymax')
        num = len(xmins)
        border = int(num/5)
        for j in range(len(xmins)):
            region = im.crop((int(xmins[j].firstChild.data), int(ymins[j].firstChild.data), int(xmaxs[j].firstChild.data), int(ymaxs[j].firstChild.data)))
            width,height = region.size
            if width<min_width:
                min_width = width
            if height<min_height:
                min_height = height
            if(j<len(xmins)-border):
                region.save("./"+"after2/train/"+type+"/"+"_"+str(j)+filelist[i])
            else:
                region.save("./"+"after2/val/"+type+"/"+"_"+str(j)+filelist[i])

In [11]:
print(min_width,min_height)

43 43


In [12]:
for type in types:
    for file in os.listdir(r"./"+"after2/train/"+type):
        im = Image.open("./"+"after2/train/"+type+"/"+file)
        out = im.resize((min_width, min_height),Image.ANTIALIAS)
        out.save("./"+"after2/train/"+type+"/"+file)
    for file in os.listdir(r"./"+"after2/val/"+type):
        im = Image.open("./"+"after2/val/"+type+"/"+file)
        out = im.resize((min_width, min_height),Image.ANTIALIAS)
        out.save("./"+"after2/val/"+type+"/"+file)

In [14]:
import os
import torch
import torch.nn as nn
import torch.optim as optim
import time
from torch.optim import lr_scheduler
from torch.autograd import Variable
from torchvision import datasets, models, transforms
import scipy.io as sio
import h5py
import numpy as np
from torchvision.datasets import ImageFolder
data_transforms = {
    'train': transforms.Compose([
        transforms.ToTensor()
    ]),
    'val': transforms.Compose([
        transforms.ToTensor()
    ]) 
}
data_dir = "./after2"
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x),
                                          data_transforms[x]) for x in ['train', 'val']}
dataloders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=1,
                                             shuffle=True, num_workers=4) for x in ['train', 'val']}
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val']}
class_names = image_datasets['train'].classes


In [15]:
dataset_sizes

{'train': 345, 'val': 27}

In [16]:
dataloders

{'train': <torch.utils.data.dataloader.DataLoader at 0x13a57952c88>,
 'val': <torch.utils.data.dataloader.DataLoader at 0x13a57952b38>}

In [22]:
from functools import reduce
import operator 

def conv3x3(in_channels, out_channels, stride=1):
    return nn.Conv2d(in_channels, out_channels, kernel_size=3,
                     stride=stride, padding=1, bias=False)


# Residual Block
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1, downsample=None):
        super(ResidualBlock, self).__init__()
        self.conv1 = conv3x3(in_channels, out_channels, stride)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(out_channels, out_channels)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.downsample = downsample

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        if self.downsample:
            residual = self.downsample(x)
        out += residual
        out = self.relu(out)
        return out


# ResNet Module
class ResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(ResNet, self).__init__()
        self.in_channels = 2
        self.conv = conv3x3(3, 2)
        self.bn = nn.BatchNorm2d(2)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(block, 2, layers[0])
        self.layer2 = self.make_layer(block, 4, layers[0], 2)
        self.layer3 = self.make_layer(block, 8, layers[1], 2)
        self.avg_pool = nn.AvgPool2d(2)
        self.fc = nn.Linear(200, num_classes)

    def make_layer(self, block, out_channels, blocks, stride=1):
        downsample = None
        if (stride != 1) or (self.in_channels != out_channels):
            downsample = nn.Sequential(
                conv3x3(self.in_channels, out_channels, stride=stride),
                nn.BatchNorm2d(out_channels))
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels
        for i in range(1, blocks):
            layers.append(block(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv(x)
        out = self.bn(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out
resnet = ResNet(ResidualBlock, [2, 2, 2, 2])
criterion = nn.CrossEntropyLoss()
lr = 0.003
optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)
for epoch in range(30):
    total = 0
    correct = 0
    counter = 0
    for phase in ['train']:
        for images,labels in dataloders[phase]:
            labels = reduce(operator.add, labels)
            # Forward + Backward + Optimize
            images,labels = Variable(images),Variable(labels)
            optimizer.zero_grad()
            outputs = resnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.unsqueeze(0).size(0)
            correct += (predicted == labels).sum()
            loss = criterion(outputs, labels.unsqueeze(0))
            loss.backward()
            counter = counter+1
            optimizer.step()
            print(phase+ "--Epoch [%d/%d], Counter: %d, Loss: %.4f, Acc: %d %%" % (epoch + 1, 2, counter,  loss.data[0], 100* correct / total))
        print(total)
        print('Accuracy: %d %%' % (100* correct / total))
        if (epoch + 1) % 20 == 0:
            lr /= 3
            optimizer = torch.optim.Adam(resnet.parameters(), lr=lr)

for phase in ['val']:
        for images,labels in dataloders[phase]:
            labels = reduce(operator.add, labels)
            # Forward + Backward + Optimize
            images,labels = Variable(images),Variable(labels)
            optimizer.zero_grad()
            outputs = resnet(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.unsqueeze(0).size(0)
            correct += (predicted == labels).sum()
            loss = criterion(outputs, labels.unsqueeze(0))
            counter = counter+1
            print(phase+ "--Epoch [%d/%d], Counter: %d, Loss: %.4f, Acc: %d %%" % (epoch + 1, 2, counter,  loss.data[0], 100* correct / total))
        print(total)
        print('Accuracy: %d %%' % (100* correct / total))
torch.save(resnet.state_dict(), 'resnet.pkl')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:94: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


train--Epoch [1/2], Counter: 1, Loss: 2.1586, Acc: 0 %
train--Epoch [1/2], Counter: 2, Loss: 2.8690, Acc: 0 %
train--Epoch [1/2], Counter: 3, Loss: 2.4757, Acc: 0 %
train--Epoch [1/2], Counter: 4, Loss: 2.9412, Acc: 0 %
train--Epoch [1/2], Counter: 5, Loss: 1.2098, Acc: 20 %
train--Epoch [1/2], Counter: 6, Loss: 2.6215, Acc: 16 %
train--Epoch [1/2], Counter: 7, Loss: 1.6127, Acc: 14 %
train--Epoch [1/2], Counter: 8, Loss: 2.6350, Acc: 12 %
train--Epoch [1/2], Counter: 9, Loss: 0.9769, Acc: 22 %
train--Epoch [1/2], Counter: 10, Loss: 2.1850, Acc: 20 %
train--Epoch [1/2], Counter: 11, Loss: 2.3440, Acc: 18 %
train--Epoch [1/2], Counter: 12, Loss: 1.1055, Acc: 25 %
train--Epoch [1/2], Counter: 13, Loss: 1.4497, Acc: 23 %
train--Epoch [1/2], Counter: 14, Loss: 2.5849, Acc: 21 %
train--Epoch [1/2], Counter: 15, Loss: 1.2841, Acc: 20 %
train--Epoch [1/2], Counter: 16, Loss: 1.0302, Acc: 25 %
train--Epoch [1/2], Counter: 17, Loss: 1.6355, Acc: 23 %
train--Epoch [1/2], Counter: 18, Loss: 2.196

train--Epoch [1/2], Counter: 172, Loss: 1.3775, Acc: 40 %
train--Epoch [1/2], Counter: 173, Loss: 1.4123, Acc: 39 %
train--Epoch [1/2], Counter: 174, Loss: 2.2680, Acc: 39 %
train--Epoch [1/2], Counter: 175, Loss: 1.4264, Acc: 39 %
train--Epoch [1/2], Counter: 176, Loss: 0.8306, Acc: 39 %
train--Epoch [1/2], Counter: 177, Loss: 1.4399, Acc: 39 %
train--Epoch [1/2], Counter: 178, Loss: 0.3809, Acc: 39 %
train--Epoch [1/2], Counter: 179, Loss: 1.6616, Acc: 39 %
train--Epoch [1/2], Counter: 180, Loss: 0.9300, Acc: 40 %
train--Epoch [1/2], Counter: 181, Loss: 2.0287, Acc: 39 %
train--Epoch [1/2], Counter: 182, Loss: 1.2111, Acc: 39 %
train--Epoch [1/2], Counter: 183, Loss: 2.4424, Acc: 39 %
train--Epoch [1/2], Counter: 184, Loss: 0.6783, Acc: 39 %
train--Epoch [1/2], Counter: 185, Loss: 1.3490, Acc: 39 %
train--Epoch [1/2], Counter: 186, Loss: 0.4439, Acc: 39 %
train--Epoch [1/2], Counter: 187, Loss: 1.4111, Acc: 39 %
train--Epoch [1/2], Counter: 188, Loss: 1.4493, Acc: 39 %
train--Epoch [

train--Epoch [1/2], Counter: 333, Loss: 0.5419, Acc: 41 %
train--Epoch [1/2], Counter: 334, Loss: 2.9132, Acc: 41 %
train--Epoch [1/2], Counter: 335, Loss: 0.3255, Acc: 41 %
train--Epoch [1/2], Counter: 336, Loss: 0.4746, Acc: 41 %
train--Epoch [1/2], Counter: 337, Loss: 1.2462, Acc: 41 %
train--Epoch [1/2], Counter: 338, Loss: 1.3399, Acc: 41 %
train--Epoch [1/2], Counter: 339, Loss: 0.4880, Acc: 41 %
train--Epoch [1/2], Counter: 340, Loss: 2.8808, Acc: 41 %
train--Epoch [1/2], Counter: 341, Loss: 1.3094, Acc: 41 %
train--Epoch [1/2], Counter: 342, Loss: 0.9215, Acc: 40 %
train--Epoch [1/2], Counter: 343, Loss: 1.2011, Acc: 40 %
train--Epoch [1/2], Counter: 344, Loss: 0.5740, Acc: 40 %
train--Epoch [1/2], Counter: 345, Loss: 2.6198, Acc: 40 %
345
Accuracy: 40 %
train--Epoch [2/2], Counter: 1, Loss: 2.3280, Acc: 0 %
train--Epoch [2/2], Counter: 2, Loss: 0.6107, Acc: 50 %
train--Epoch [2/2], Counter: 3, Loss: 0.9481, Acc: 33 %
train--Epoch [2/2], Counter: 4, Loss: 2.6229, Acc: 25 %
trai

train--Epoch [2/2], Counter: 151, Loss: 1.7656, Acc: 34 %
train--Epoch [2/2], Counter: 152, Loss: 0.9095, Acc: 34 %
train--Epoch [2/2], Counter: 153, Loss: 0.9172, Acc: 35 %
train--Epoch [2/2], Counter: 154, Loss: 1.0349, Acc: 35 %
train--Epoch [2/2], Counter: 155, Loss: 0.7723, Acc: 36 %
train--Epoch [2/2], Counter: 156, Loss: 0.5058, Acc: 36 %
train--Epoch [2/2], Counter: 157, Loss: 0.6589, Acc: 36 %
train--Epoch [2/2], Counter: 158, Loss: 0.5447, Acc: 37 %
train--Epoch [2/2], Counter: 159, Loss: 2.0434, Acc: 37 %
train--Epoch [2/2], Counter: 160, Loss: 1.8288, Acc: 36 %
train--Epoch [2/2], Counter: 161, Loss: 1.8179, Acc: 36 %
train--Epoch [2/2], Counter: 162, Loss: 1.7860, Acc: 36 %
train--Epoch [2/2], Counter: 163, Loss: 0.4472, Acc: 36 %
train--Epoch [2/2], Counter: 164, Loss: 0.6612, Acc: 37 %
train--Epoch [2/2], Counter: 165, Loss: 0.4544, Acc: 37 %
train--Epoch [2/2], Counter: 166, Loss: 0.2878, Acc: 37 %
train--Epoch [2/2], Counter: 167, Loss: 0.3321, Acc: 38 %
train--Epoch [

...

train--Epoch [29/2], Counter: 155, Loss: 0.1230, Acc: 96 %
train--Epoch [29/2], Counter: 156, Loss: 0.0913, Acc: 96 %
train--Epoch [29/2], Counter: 157, Loss: 0.5979, Acc: 96 %
train--Epoch [29/2], Counter: 158, Loss: 0.0901, Acc: 96 %
train--Epoch [29/2], Counter: 159, Loss: 0.4060, Acc: 96 %
train--Epoch [29/2], Counter: 160, Loss: 0.2789, Acc: 96 %
train--Epoch [29/2], Counter: 161, Loss: 0.0052, Acc: 96 %
train--Epoch [29/2], Counter: 162, Loss: 0.0384, Acc: 96 %
train--Epoch [29/2], Counter: 163, Loss: 0.0616, Acc: 96 %
train--Epoch [29/2], Counter: 164, Loss: 0.3668, Acc: 96 %
train--Epoch [29/2], Counter: 165, Loss: 0.6972, Acc: 96 %
train--Epoch [29/2], Counter: 166, Loss: 0.5497, Acc: 96 %
train--Epoch [29/2], Counter: 167, Loss: 0.1484, Acc: 96 %
train--Epoch [29/2], Counter: 168, Loss: 0.5051, Acc: 96 %
train--Epoch [29/2], Counter: 169, Loss: 0.4339, Acc: 96 %
train--Epoch [29/2], Counter: 170, Loss: 0.2759, Acc: 96 %
train--Epoch [29/2], Counter: 171, Loss: 0.8423, Acc: 95

train--Epoch [29/2], Counter: 308, Loss: 0.3692, Acc: 92 %
train--Epoch [29/2], Counter: 309, Loss: 0.4116, Acc: 92 %
train--Epoch [29/2], Counter: 310, Loss: 0.1816, Acc: 92 %
train--Epoch [29/2], Counter: 311, Loss: 0.0782, Acc: 92 %
train--Epoch [29/2], Counter: 312, Loss: 0.1129, Acc: 92 %
train--Epoch [29/2], Counter: 313, Loss: 0.3105, Acc: 92 %
train--Epoch [29/2], Counter: 314, Loss: 0.3818, Acc: 92 %
train--Epoch [29/2], Counter: 315, Loss: 0.3258, Acc: 93 %
train--Epoch [29/2], Counter: 316, Loss: 0.0090, Acc: 93 %
train--Epoch [29/2], Counter: 317, Loss: 0.0337, Acc: 93 %
train--Epoch [29/2], Counter: 318, Loss: 0.0104, Acc: 93 %
train--Epoch [29/2], Counter: 319, Loss: 0.1778, Acc: 93 %
train--Epoch [29/2], Counter: 320, Loss: 0.0087, Acc: 93 %
train--Epoch [29/2], Counter: 321, Loss: 0.0281, Acc: 93 %
train--Epoch [29/2], Counter: 322, Loss: 0.3533, Acc: 93 %
train--Epoch [29/2], Counter: 323, Loss: 0.1437, Acc: 93 %
train--Epoch [29/2], Counter: 324, Loss: 0.0292, Acc: 93

train--Epoch [30/2], Counter: 104, Loss: 0.0075, Acc: 97 %
train--Epoch [30/2], Counter: 105, Loss: 0.0827, Acc: 97 %
train--Epoch [30/2], Counter: 106, Loss: 0.2206, Acc: 97 %
train--Epoch [30/2], Counter: 107, Loss: 0.1334, Acc: 97 %
train--Epoch [30/2], Counter: 108, Loss: 0.2100, Acc: 97 %
train--Epoch [30/2], Counter: 109, Loss: 0.0522, Acc: 97 %
train--Epoch [30/2], Counter: 110, Loss: 0.3349, Acc: 97 %
train--Epoch [30/2], Counter: 111, Loss: 0.0830, Acc: 97 %
train--Epoch [30/2], Counter: 112, Loss: 0.0732, Acc: 97 %
train--Epoch [30/2], Counter: 113, Loss: 0.5908, Acc: 97 %
train--Epoch [30/2], Counter: 114, Loss: 0.0279, Acc: 97 %
train--Epoch [30/2], Counter: 115, Loss: 0.0028, Acc: 97 %
train--Epoch [30/2], Counter: 116, Loss: 0.4165, Acc: 97 %
train--Epoch [30/2], Counter: 117, Loss: 0.0854, Acc: 97 %
train--Epoch [30/2], Counter: 118, Loss: 0.0033, Acc: 97 %
train--Epoch [30/2], Counter: 119, Loss: 0.1238, Acc: 97 %
train--Epoch [30/2], Counter: 120, Loss: 0.1598, Acc: 97

train--Epoch [30/2], Counter: 262, Loss: 0.2232, Acc: 94 %
train--Epoch [30/2], Counter: 263, Loss: 0.1178, Acc: 94 %
train--Epoch [30/2], Counter: 264, Loss: 0.1878, Acc: 94 %
train--Epoch [30/2], Counter: 265, Loss: 0.0135, Acc: 94 %
train--Epoch [30/2], Counter: 266, Loss: 0.6054, Acc: 94 %
train--Epoch [30/2], Counter: 267, Loss: 0.0937, Acc: 94 %
train--Epoch [30/2], Counter: 268, Loss: 0.2260, Acc: 94 %
train--Epoch [30/2], Counter: 269, Loss: 0.6515, Acc: 94 %
train--Epoch [30/2], Counter: 270, Loss: 0.1007, Acc: 94 %
train--Epoch [30/2], Counter: 271, Loss: 0.0452, Acc: 94 %
train--Epoch [30/2], Counter: 272, Loss: 0.9947, Acc: 94 %
train--Epoch [30/2], Counter: 273, Loss: 0.3117, Acc: 94 %
train--Epoch [30/2], Counter: 274, Loss: 0.1051, Acc: 94 %
train--Epoch [30/2], Counter: 275, Loss: 0.0262, Acc: 94 %
train--Epoch [30/2], Counter: 276, Loss: 0.0093, Acc: 94 %
train--Epoch [30/2], Counter: 277, Loss: 0.0088, Acc: 94 %
train--Epoch [30/2], Counter: 278, Loss: 0.0495, Acc: 94

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:113: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
